In [ ]:
!pip install catboost
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time
import sys
import re
import math
import numpy as np
import pandas as pd
import xlwt
import random
import os
from os import path
from google.colab import drive

In [ ]:
#구글 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
daily_info = pd.read_csv('/content/drive/MyDrive/11st_crawling/mining.csv')
daily_info

,Unnamed: 0,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
0,0,0,2.797325e+09,[메리바움]41%쿠폰+T22% 신상득템 예쁜옷가득! 데일리룩&홈웨어 맨투맨/셔츠/팬...,2021-02-28,여성의류,여성의류,블라우스/셔츠,1.0,58068,5일 02시간,0,4707.0,11900,14,MERRYBAUM,NaN,고객서비스 우수,61.5,100.0,5.0,4.0,42%,28%,19%,6%,5%,"마감,사이즈,색상,배송속도","40%,49%,77%,90%","깔끔해요,정사이즈에요,화면대로에요,보통이에요","가격320,구매35,디자인447,사용성368,상품평2,색상181,서비스4,성능3,양...",5,28,6
1,1,1,2.661369e+09,[온더리버]41%+T22% 봄신상할인! 데일리&홈웨어 득템! 맨투맨/세트/원피스/자...,2021-02-28,여성의류,여성의류,원피스,2.0,104087,5일 02시간,0,19310.0,9900,16,[온더리버],NaN,고객서비스 우수,63.7,100.0,5.0,3.9,42%,26%,20%,6%,6%,"마감,사이즈,색상,배송속도","41%,46%,80%,89%","깔끔해요,정사이즈에요,화면대로에요,보통이에요","가격1,310,구매116,디자인2,358,무게2,브랜드6,사용성1,602,상품평16...",5,28,6
2,2,2,6.093780e+07,[요즘에]31%쿠폰+T11%모든체형 날씬핏&넘편한 팬츠NO.1! 데님/밴딩/슬랙스/...,2021-02-28,여성의류,여성의류,청바지/팬츠,3.0,67434,5일 02시간,0,150332.0,9900,23,요즘에,NaN,고객서비스 우수,86.9,100.0,5.0,4.1,49%,24%,18%,5%,4%,"마감,사이즈,색상,배송속도","55%,49%,75%,91%","깔끔해요,정사이즈에요,화면대로에요,빨라요","가격11,248,구매198,냄새1,디자인7,722,무게5,브랜드5,사용성5,673,...",5,28,6
3,3,3,3.221037e+09,"[쇼인더룸]11%+20% 5,990/9,840~특가 맨투맨/빅사이즈/봄신상",2021-02-28,여성의류,여성의류,티셔츠,4.0,4040,3일 02시간,0,253.0,13800,-1,쇼인더룸,NaN,고객서비스 우수,100.0,98.4,3.0,3.8,37%,30%,16%,8%,9%,"마감,사이즈,색상,배송속도","42%,56%,77%,87%","깔끔해요,정사이즈에요,화면대로에요,빨라요","가격13,디자인21,사용성23,상품평1,색상6,서비스1,양1,재질1,제품일반47,크...",3,28,6
4,4,4,1.695439e+09,[에이와비]15%+15% 신상업뎃완료!키커보이고 길어보이는 키작녀쇼핑몰!후기극찬!메...,2021-02-28,여성의류,여성의류,원피스,5.0,19928,5일 02시간,0,26417.0,14900,16,에이와비,NaN,고객서비스 우수,83.3,100.0,4.0,4.1,45%,29%,18%,5%,3%,"마감,사이즈,색상,배송속도","42%,54%,77%,89%","깔끔해요,정사이즈에요,화면대로에요,보통이에요","가격993,구매65,디자인3,472,무게3,브랜드1,사용성1,752,상품평16,색상...",5,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527890,527890,698764,3.008019e+09,소노벨비발디파크펫룸(구 비발디파크홍천)애완동반/애완/강원도리조트/홍천대명/대명리조트...,2021-05-19,여행/티켓,국내숙박,리조트,1.0,9,7일 02시간,1,2.0,234000,1,콘도24닷컴,NaN,NaN,90.9,100.0,4.0,5.0,100%,0%,0%,0%,0%,"객실청결도,객실내부시설,교통,직원친절도","100%,100%,100%,100%","깨끗해요,만족해요,편리해요,친절해요",NaN,7,19,2
527891,527891,698765,3.487145e+09,[기프티콘] 던킨 도넛 10개팩,2021-05-19,e쿠폰/상품권,카페/베이커리,베이커리/도넛,1.0,816,5일 02시간,1,8.0,15000,-1,기프티콘_11번가,NaN,NaN,95.5,99.9,5.0,4.9,87%,13%,0%,0%,0%,NaN,NaN,NaN,NaN,5,19,2
527892,527892,698766,2.240799e+09,워터클린 에어컨 청소 [구매 후 예약 필수],2021-05-19,e쿠폰/상품권,영화/교육/이용권,서비스 선택,1.0,1909,2일 02시간,1,1768.0,34900,-1,주식회사_워터클린,NaN,NaN,94.6,100.0,3.0,4.8,85%,10%,3%,1%,1%,"서비스가격,서비스만족도,시간약속","69%,63%,81%","저렴해요,만족해요,맞춰왔어요","가격95,디자인1,사용성391,상품평8,색상1,선호도1,양80,제품일반365,크기1...",2,19,2
527893,527893,698768,3.486517e+09,이마트/트레이더스 금액권 5만원권,2021-05-19,e쿠폰/상품권,상품권/마트/면세점,-1,1.0,12782,6일 02시간,1,1247.0,47000,6,기프티콘_11번가,NaN,NaN,95.5,99.9,5.0,4.9,89%,8%,1%,1%,1%,NaN,NaN,NaN,"가격61,구매73,사용성32,양10,제품일반249,크기6,품질16",6,19,2


In [ ]:
daily_info['product number'].value_counts()

8.320366e+08    89
1.874880e+09    88
2.430953e+09    87
2.059463e+09    87
2.798218e+09    87
                ..
3.230462e+09     1
3.230584e+09     1
3.230640e+09     1
3.485110e+09     1
3.362363e+09     1
Name: product number, Length: 24117, dtype: int64

In [ ]:
type(daily_info['total_sales'][0])

numpy.int64

In [ ]:
daily_info.describe()

,Unnamed: 0,index,product number,sales_rank,total_sales,free_delivery,review_num,price,discount_per,official_store,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
count,527895.000000,527895.000000,5.278950e+05,527895.000000,527895.000000,527895.000000,527895.000000,5.278950e+05,527895.000000,180041.0,527895.000000,527895.000000,519879.000000,527029.000000,527895.000000,527895.000000,527895.000000
mean,263947.000000,350861.654812,2.436337e+09,96.056451,1497.707105,0.322655,1701.518103,8.344736e+04,7.021296,1.0,86.669193,91.087199,3.632461,4.552238,6.490656,15.249665,3.002491
std,152390.304514,200883.026930,8.610894e+08,85.201123,7652.422401,0.467492,9074.273382,2.787282e+05,12.430748,0.0,19.444976,25.252810,0.862203,0.389834,3.769351,8.569684,2.035882
min,0.000000,0.000000,4.250300e+04,1.000000,-187.000000,0.000000,0.000000,2.000000e+00,-1.000000,1.0,-1.000000,-1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,131973.500000,178077.500000,1.973060e+09,31.000000,8.000000,0.000000,4.000000,9.900000e+03,-1.000000,1.0,85.900000,98.000000,3.000000,4.400000,3.000000,8.000000,1.000000
50%,263947.000000,348921.000000,2.692975e+09,72.000000,57.000000,0.000000,47.000000,1.690000e+04,1.000000,1.0,91.600000,100.000000,4.000000,4.600000,6.000000,15.000000,3.000000
75%,395920.500000,525690.500000,3.110339e+09,137.000000,428.000000,1.000000,441.000000,3.600000e+04,11.000000,1.0,95.500000,100.000000,4.000000,4.800000,10.000000,23.000000,5.000000
max,527894.000000,698769.000000,3.506492e+09,666.000000,705017.000000,1.000000,388775.000000,5.806000e+06,100.000000,1.0,100.000000,100.000000,5.000000,5.000000,15.000000,31.000000,6.000000


In [ ]:
#daily_info = daily_info[daily_info['product_large_type'] == '여성의류']
#daily_info = daily_info[daily_info['product_medium_type'] == '노트북/PC/태블릿']
#daily_info = daily_info[daily_info['product_medium_type'] == '신선식품']
daily_info = daily_info[daily_info['product_large_type'] == '디지털']
#daily_info = daily_info[daily_info['product_medium_type'] == '남성의류']
#daily_info = daily_info[daily_info['product_large_type'] == '뷰티']
daily_info

,Unnamed: 0,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
4970,4970,6233,2.831097e+09,삼성 갤럭시탭 s6 라이트 S펜수납 북커버 케이스,2021-02-28,디지털,노트북/PC/태블릿,액세서리,1.0,1896,4일 02시간,1,277.0,29700,15,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,78%,16%,4%,1%,1%,"성능,무게,배터리,배송속도","84%,89%,55%,50%","좋아요,보통이에요,보통이에요,빨라요","가격21,디자인45,사용성60,상품평4,색상44,성능9,양7,제품일반105,크기4,...",4,28,6
4971,4971,6234,2.247584e+09,Apple Pencil 애플 펜슬 (2세대) [MU8F2KH/A] iPad Pro ...,2021-02-28,디지털,노트북/PC/태블릿,액세서리,2.0,3624,6일 02시간,0,896.0,165000,-1,Apple공인인증판매점,1.0,고객서비스 우수,89.6,100.0,5.0,4.8,86%,10%,2%,1%,1%,"성능,무게,배터리,배송속도","75%,93%,55%,59%","좋아요,가벼워요,오래가요,빨라요","가격75,구매9,디자인15,사용성90,색상1,서비스2,성능23,양3,제품일반257,...",6,28,6
4972,4972,6235,3.012948e+09,1+1 삼성 갤럭시탭 s7 북커버 케이스 S펜 수납,2021-02-28,디지털,노트북/PC/태블릿,액세서리,3.0,827,2일 02시간,1,104.0,29700,12,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,74%,24%,2%,0%,0%,"성능,무게,배터리,배송속도","87%,90%,56%,51%","좋아요,보통이에요,보통이에요,빨라요","가격5,디자인13,사용성14,상품평1,색상12,서비스1,성능1,양3,제품일반39,크...",2,28,6
4973,4973,6236,2.820897e+09,인증점 삼성 갤럭시탭S6 라이트 lite 10.4 SM-P610/P615 WIFI/...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,4.0,1520,10일 02시간,0,425.0,428450,5,삼성전자공식파트너,1.0,고객서비스 우수,90.2,100.0,4.0,4.8,79%,16%,3%,1%,1%,"성능,무게,배터리,배송속도","65%,85%,57%,56%","좋아요,보통이에요,오래가요,빨라요","가격30,구매4,디자인27,무게2,사용성80,색상16,서비스1,성능34,양6,재질1...",10,28,6
4974,4974,6237,2.961319e+09,삼성 갤럭시탭S7플러스 12.4 SM-T975 256G LTE 갤럭시탭S7+/아카데...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,5.0,887,3일 02시간,0,175.0,1174630,5,삼성공식파트너엠피나비,1.0,고객서비스 우수,80.1,100.0,4.0,4.8,83%,15%,1%,1%,0%,"성능,무게,배터리,배송속도","68%,97%,51%,58%","좋아요,보통이에요,오래가요,빨라요","가격6,구매2,디자인6,무게2,사용성26,색상2,성능16,양13,제품일반48,크기8...",3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527294,527294,697993,3.234159e+09,"+최종혜택 210,120 / 오늘출발+삼성전자 SL-T1672W 정품무한 잉크젯 복...",2021-05-19,디지털,모니터/프린터,프린터/복합기,122.0,206,14일 01시간,0,77.0,229000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.8,82%,17%,1%,0%,0%,NaN,NaN,NaN,"가격8,디자인2,무게1,사용성11,색상2,서비스1,설치2,성능2,양6,제품일반38,크기6",14,19,2
527295,527295,697994,2.904190e+09,"+최종혜택 155,060 / 오늘출발+삼성전자 SL-C513 컬러 레이저 프린터기 ...",2021-05-19,디지털,모니터/프린터,프린터/복합기,123.0,47,14일 01시간,0,52.0,169000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.5,61%,29%,8%,0%,2%,"인쇄품질,인쇄속도,배송속도","69%,84%,61%","선명해요,빨라요,빨라요","가격4,사용성5,서비스1,설치1,양1,제품일반17,크기2,품질4",14,19,2
527296,527296,697995,2.904112e+09,"+최종혜택 283,510 / 오늘출발+삼성전자 SL-C483W 컬러 레이저 복합기/...",2021-05-19,디지털,모니터/프린터,프린터/복합기,124.0,30,14일 01시간,0,20.0,309000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.4,55%,35%,5%,5%,0%,"인쇄품질,인쇄속도,배송속도","72%,94%,67%","선명해요,빨라요,빨라요","가격1,사용성1,색상1,서비스2,성능1,제품일반6,품질2",14,19,2
527297,527297,697996,2.904196e+09,"+최종혜택 200,940+삼성전자 SL-C513W 컬러 레이저 프린터기 정품토너포함...",2021-05-19,디지털,모니터/프린터,프린터/복합기,125.0,71,14일 01시간,0,49.0,219000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.6,70%,20%,10%,0%,0%,"인쇄품질,인쇄속도,배송속도","81%,76%,71%","선명해요,빨라요,빨라요","가격4,브랜드1,사용성7,색상2,서비스3,양1,제품일반18,품질4",14,19,2


In [ ]:
#daily_info = daily_info[daily_info['sales_rank'] <= 50]
#daily_info = daily_info[daily_info['sales_rank'] <= 70]
daily_info

,Unnamed: 0,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
4970,4970,6233,2.831097e+09,삼성 갤럭시탭 s6 라이트 S펜수납 북커버 케이스,2021-02-28,디지털,노트북/PC/태블릿,액세서리,1.0,1896,4일 02시간,1,277.0,29700,15,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,78%,16%,4%,1%,1%,"성능,무게,배터리,배송속도","84%,89%,55%,50%","좋아요,보통이에요,보통이에요,빨라요","가격21,디자인45,사용성60,상품평4,색상44,성능9,양7,제품일반105,크기4,...",4,28,6
4971,4971,6234,2.247584e+09,Apple Pencil 애플 펜슬 (2세대) [MU8F2KH/A] iPad Pro ...,2021-02-28,디지털,노트북/PC/태블릿,액세서리,2.0,3624,6일 02시간,0,896.0,165000,-1,Apple공인인증판매점,1.0,고객서비스 우수,89.6,100.0,5.0,4.8,86%,10%,2%,1%,1%,"성능,무게,배터리,배송속도","75%,93%,55%,59%","좋아요,가벼워요,오래가요,빨라요","가격75,구매9,디자인15,사용성90,색상1,서비스2,성능23,양3,제품일반257,...",6,28,6
4972,4972,6235,3.012948e+09,1+1 삼성 갤럭시탭 s7 북커버 케이스 S펜 수납,2021-02-28,디지털,노트북/PC/태블릿,액세서리,3.0,827,2일 02시간,1,104.0,29700,12,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,74%,24%,2%,0%,0%,"성능,무게,배터리,배송속도","87%,90%,56%,51%","좋아요,보통이에요,보통이에요,빨라요","가격5,디자인13,사용성14,상품평1,색상12,서비스1,성능1,양3,제품일반39,크...",2,28,6
4973,4973,6236,2.820897e+09,인증점 삼성 갤럭시탭S6 라이트 lite 10.4 SM-P610/P615 WIFI/...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,4.0,1520,10일 02시간,0,425.0,428450,5,삼성전자공식파트너,1.0,고객서비스 우수,90.2,100.0,4.0,4.8,79%,16%,3%,1%,1%,"성능,무게,배터리,배송속도","65%,85%,57%,56%","좋아요,보통이에요,오래가요,빨라요","가격30,구매4,디자인27,무게2,사용성80,색상16,서비스1,성능34,양6,재질1...",10,28,6
4974,4974,6237,2.961319e+09,삼성 갤럭시탭S7플러스 12.4 SM-T975 256G LTE 갤럭시탭S7+/아카데...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,5.0,887,3일 02시간,0,175.0,1174630,5,삼성공식파트너엠피나비,1.0,고객서비스 우수,80.1,100.0,4.0,4.8,83%,15%,1%,1%,0%,"성능,무게,배터리,배송속도","68%,97%,51%,58%","좋아요,보통이에요,오래가요,빨라요","가격6,구매2,디자인6,무게2,사용성26,색상2,성능16,양13,제품일반48,크기8...",3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527294,527294,697993,3.234159e+09,"+최종혜택 210,120 / 오늘출발+삼성전자 SL-T1672W 정품무한 잉크젯 복...",2021-05-19,디지털,모니터/프린터,프린터/복합기,122.0,206,14일 01시간,0,77.0,229000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.8,82%,17%,1%,0%,0%,NaN,NaN,NaN,"가격8,디자인2,무게1,사용성11,색상2,서비스1,설치2,성능2,양6,제품일반38,크기6",14,19,2
527295,527295,697994,2.904190e+09,"+최종혜택 155,060 / 오늘출발+삼성전자 SL-C513 컬러 레이저 프린터기 ...",2021-05-19,디지털,모니터/프린터,프린터/복합기,123.0,47,14일 01시간,0,52.0,169000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.5,61%,29%,8%,0%,2%,"인쇄품질,인쇄속도,배송속도","69%,84%,61%","선명해요,빨라요,빨라요","가격4,사용성5,서비스1,설치1,양1,제품일반17,크기2,품질4",14,19,2
527296,527296,697995,2.904112e+09,"+최종혜택 283,510 / 오늘출발+삼성전자 SL-C483W 컬러 레이저 복합기/...",2021-05-19,디지털,모니터/프린터,프린터/복합기,124.0,30,14일 01시간,0,20.0,309000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.4,55%,35%,5%,5%,0%,"인쇄품질,인쇄속도,배송속도","72%,94%,67%","선명해요,빨라요,빨라요","가격1,사용성1,색상1,서비스2,성능1,제품일반6,품질2",14,19,2
527297,527297,697996,2.904196e+09,"+최종혜택 200,940+삼성전자 SL-C513W 컬러 레이저 프린터기 정품토너포함...",2021-05-19,디지털,모니터/프린터,프린터/복합기,125.0,71,14일 01시간,0,49.0,219000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.6,70%,20%,10%,0%,0%,"인쇄품질,인쇄속도,배송속도","81%,76%,71%","선명해요,빨라요,빨라요","가격4,브랜드1,사용성7,색상2,서비스3,양1,제품일반18,품질4",14,19,2


In [ ]:
daily_info.corr()['sales_rank'].sort_values(ascending = False)

sales_rank                     1.000000
remain_sales_limit             0.269775
product number                 0.157654
free_delivery                  0.133197
Unnamed: 0                     0.033285
index                          0.033266
discount_per                   0.024007
weekday                        0.020112
supplier_sales_performance     0.004124
day                           -0.006993
price                         -0.039616
review_num                    -0.087774
supplier_response_per         -0.097162
supplier_pos_review           -0.099093
customer_total_satisfaction   -0.158032
total_sales                   -0.170105
official_store                      NaN
Name: sales_rank, dtype: float64

In [ ]:
daily_info.corr(method = 'spearman')['sales_rank'].sort_values(ascending = False)

sales_rank                     1.000000
remain_sales_limit             0.279099
product number                 0.157498
free_delivery                  0.125038
discount_per                   0.053282
index                          0.037543
Unnamed: 0                     0.037543
weekday                        0.015236
supplier_sales_performance     0.013682
day                           -0.009384
supplier_response_per         -0.028124
customer_total_satisfaction   -0.049409
price                         -0.154089
supplier_pos_review           -0.182764
review_num                    -0.344333
total_sales                   -0.374228
official_store                      NaN
Name: sales_rank, dtype: float64

In [ ]:
daily_info['product_small_type'].value_counts()

케이스/보호필름     13680
프린터/복합기       7904
저장장치/USB      6733
스마트폰액세서리      6508
액세서리          5570
태블릿           5343
배터리/충전용품      2475
키보드/마우스       2402
가입상품/공기계      1873
노트북           1611
공유기/IP카메라     1204
모니터/주변기기      1088
PC주변용품/SW      767
데스크탑/조립PC      613
스피커/헤드셋        585
게임기/타이틀        463
잉크/토너          107
PC부품            45
Name: product_small_type, dtype: int64

In [ ]:
daily_info = daily_info.drop(columns = ['Unnamed: 0'])

In [ ]:
daily_info.reset_index(drop = True, inplace = True)

In [ ]:
a = daily_info[daily_info['date'] == '2021-05-19'].index
a

Int64Index([58264, 58265, 58266, 58267, 58268, 58269, 58270, 58271, 58272,
            58273,
            ...
            58961, 58962, 58963, 58964, 58965, 58966, 58967, 58968, 58969,
            58970],
           dtype='int64', length=707)

In [ ]:
train = daily_info[:a[0]]
train

,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
0,6233,2.831097e+09,삼성 갤럭시탭 s6 라이트 S펜수납 북커버 케이스,2021-02-28,디지털,노트북/PC/태블릿,액세서리,1.0,1896,4일 02시간,1,277.0,29700,15,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,78%,16%,4%,1%,1%,"성능,무게,배터리,배송속도","84%,89%,55%,50%","좋아요,보통이에요,보통이에요,빨라요","가격21,디자인45,사용성60,상품평4,색상44,성능9,양7,제품일반105,크기4,...",4,28,6
1,6234,2.247584e+09,Apple Pencil 애플 펜슬 (2세대) [MU8F2KH/A] iPad Pro ...,2021-02-28,디지털,노트북/PC/태블릿,액세서리,2.0,3624,6일 02시간,0,896.0,165000,-1,Apple공인인증판매점,1.0,고객서비스 우수,89.6,100.0,5.0,4.8,86%,10%,2%,1%,1%,"성능,무게,배터리,배송속도","75%,93%,55%,59%","좋아요,가벼워요,오래가요,빨라요","가격75,구매9,디자인15,사용성90,색상1,서비스2,성능23,양3,제품일반257,...",6,28,6
2,6235,3.012948e+09,1+1 삼성 갤럭시탭 s7 북커버 케이스 S펜 수납,2021-02-28,디지털,노트북/PC/태블릿,액세서리,3.0,827,2일 02시간,1,104.0,29700,12,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,74%,24%,2%,0%,0%,"성능,무게,배터리,배송속도","87%,90%,56%,51%","좋아요,보통이에요,보통이에요,빨라요","가격5,디자인13,사용성14,상품평1,색상12,서비스1,성능1,양3,제품일반39,크...",2,28,6
3,6236,2.820897e+09,인증점 삼성 갤럭시탭S6 라이트 lite 10.4 SM-P610/P615 WIFI/...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,4.0,1520,10일 02시간,0,425.0,428450,5,삼성전자공식파트너,1.0,고객서비스 우수,90.2,100.0,4.0,4.8,79%,16%,3%,1%,1%,"성능,무게,배터리,배송속도","65%,85%,57%,56%","좋아요,보통이에요,오래가요,빨라요","가격30,구매4,디자인27,무게2,사용성80,색상16,서비스1,성능34,양6,재질1...",10,28,6
4,6237,2.961319e+09,삼성 갤럭시탭S7플러스 12.4 SM-T975 256G LTE 갤럭시탭S7+/아카데...,2021-02-28,디지털,노트북/PC/태블릿,태블릿,5.0,887,3일 02시간,0,175.0,1174630,5,삼성공식파트너엠피나비,1.0,고객서비스 우수,80.1,100.0,4.0,4.8,83%,15%,1%,1%,0%,"성능,무게,배터리,배송속도","68%,97%,51%,58%","좋아요,보통이에요,오래가요,빨라요","가격6,구매2,디자인6,무게2,사용성26,색상2,성능16,양13,제품일반48,크기8...",3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58259,688588,1.374893e+09,"[최종혜택 347,750 / 오늘출발] (JU) SL-C483FW 컬러 삼성복합기 ...",2021-05-18,디지털,모니터/프린터,프린터/복합기,114.0,5,1일 13시간,0,78.0,379000,2,삼성공식파트너,1.0,고객서비스 우수,96.0,100.0,4.0,4.4,62%,24%,10%,1%,3%,"인쇄품질,인쇄속도,배송속도","74%,78%,56%","선명해요,빨라요,빨라요","가격2,디자인1,사용성3,성능2,제품일반13,크기2,품질3",1,18,1
58260,688589,1.323999e+09,"[최종혜택 164,240] 삼성전자 SL-M2077F 토너포함 흑백레이저팩스복합기/...",2021-05-18,디지털,모니터/프린터,프린터/복합기,115.0,2,6일 02시간,0,406.0,179000,2,삼성공식파트너_디지탈태양,1.0,고객서비스 우수,95.2,100.0,4.0,4.7,78%,14%,6%,1%,1%,"인쇄품질,인쇄속도,배송속도","78%,73%,63%","선명해요,빨라요,빨라요","가격21,디자인2,사용성6,서비스1,설치1,성능14,양1,제품일반54,크기1,품질5",6,18,1
58261,688591,2.904099e+09,"+최종혜택 200,940 / 오늘출발+삼성전자 SL-C433W 컬러 레이저 프린터기...",2021-05-18,디지털,모니터/프린터,프린터/복합기,117.0,13,1일 02시간,0,7.0,219000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.6,72%,14%,14%,0%,0%,"인쇄품질,인쇄속도,배송속도","57%,86%,100%","선명해요,빨라요,빨라요","색상1,제품일반5",1,18,1
58262,688592,2.630368e+09,"[최종혜택 90,840 / 오늘출발] (JU) SL-M2030 재택근무 흑백 삼성프...",2021-05-18,디지털,모니터/프린터,프린터/복합기,118.0,41,1일 13시간,0,87.0,99000,2,삼성공식파트너,1.0,고객서비스 우수,96.0,100.0,4.0,4.9,90%,8%,1%,0%,1%,"인쇄품질,인쇄속도,배송속도","91%,92%,86%","선명해요,빨라요,빨라요","가격19,구매5,디자인2,사용성23,색상1,서비스2,설치1,성능5,양3,제품일반43...",1,18,1


In [ ]:
test = daily_info[a[0]:]
test

,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
58264,697041,3.486486e+09,최종64만/한컴+마우스/HP 15s-fq2013TU 가성비 노트북/11세대 i5/2...,2021-05-19,디지털,노트북/PC/태블릿,노트북,1.0,234,0일 0시간,0,0.0,760000,5,HP총판이텍컴퓨터,1.0,고객서비스 우수,95.5,97.5,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,2
58265,697042,2.884596e+09,고부기 갤럭시 워치 액티브2 3D 포밍 풀커버 액정보호필름 2매,2021-05-19,디지털,노트북/PC/태블릿,액세서리,2.0,293,3일 01시간,1,107.0,8900,10,고부기몰,1.0,고객서비스 우수,88.4,100.0,4.0,4.4,59%,29%,7%,4%,1%,"성능,무게,배터리,배송속도","73%,66%,64%,49%","좋아요,가벼워요,보통이에요,빨라요","사용성8,제품일반23,크기1,품질3",3,19,2
58266,697043,2.817019e+09,[쿠폰15%]공식파트너 삼성 갤럭시탭S6 라이트 64GB 10.4 SM-P610 W...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,3.0,1019,3일 01시간,0,563.0,405900,10,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,4.6,72%,22%,4%,1%,1%,"성능,무게,배터리,배송속도","70%,84%,61%,54%","좋아요,보통이에요,오래가요,빨라요","가격57,구매12,디자인31,사용성107,상품평3,색상25,선호도1,성능34,양15...",3,19,2
58267,697044,3.411387e+09,[쿠폰16%]공식파트너 삼성 갤럭시탭S7 네이비 11.0 SM-T870 512G L...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,4.0,403,1일 01시간,0,292.0,999400,-1,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,5.0,95%,4%,1%,0%,0%,"성능,무게,배터리,배송속도","92%,99%,56%,72%","좋아요,보통이에요,오래가요,빨라요","가격20,구매9,디자인29,무게2,사용성81,색상30,성능49,양36,일반1,제품일...",1,19,2
58268,697045,2.540957e+09,[쿠폰15%할인]공식파트너 삼성 갤럭시탭A 8.0 2019 SM-T295 32GB/...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,5.0,484,1일 01시간,0,206.0,168300,10,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,4.6,67%,21%,10%,1%,1%,"성능,무게,배터리,배송속도","83%,68%,60%,50%","좋아요,보통이에요,오래가요,빨라요","가격29,구매2,디자인4,무게2,사용성37,상품평1,성능18,양3,제품일반62,크기...",1,19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58966,697993,3.234159e+09,"+최종혜택 210,120 / 오늘출발+삼성전자 SL-T1672W 정품무한 잉크젯 복...",2021-05-19,디지털,모니터/프린터,프린터/복합기,122.0,206,14일 01시간,0,77.0,229000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.8,82%,17%,1%,0%,0%,NaN,NaN,NaN,"가격8,디자인2,무게1,사용성11,색상2,서비스1,설치2,성능2,양6,제품일반38,크기6",14,19,2
58967,697994,2.904190e+09,"+최종혜택 155,060 / 오늘출발+삼성전자 SL-C513 컬러 레이저 프린터기 ...",2021-05-19,디지털,모니터/프린터,프린터/복합기,123.0,47,14일 01시간,0,52.0,169000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.5,61%,29%,8%,0%,2%,"인쇄품질,인쇄속도,배송속도","69%,84%,61%","선명해요,빨라요,빨라요","가격4,사용성5,서비스1,설치1,양1,제품일반17,크기2,품질4",14,19,2
58968,697995,2.904112e+09,"+최종혜택 283,510 / 오늘출발+삼성전자 SL-C483W 컬러 레이저 복합기/...",2021-05-19,디지털,모니터/프린터,프린터/복합기,124.0,30,14일 01시간,0,20.0,309000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.4,55%,35%,5%,5%,0%,"인쇄품질,인쇄속도,배송속도","72%,94%,67%","선명해요,빨라요,빨라요","가격1,사용성1,색상1,서비스2,성능1,제품일반6,품질2",14,19,2
58969,697996,2.904196e+09,"+최종혜택 200,940+삼성전자 SL-C513W 컬러 레이저 프린터기 정품토너포함...",2021-05-19,디지털,모니터/프린터,프린터/복합기,125.0,71,14일 01시간,0,49.0,219000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.6,70%,20%,10%,0%,0%,"인쇄품질,인쇄속도,배송속도","81%,76%,71%","선명해요,빨라요,빨라요","가격4,브랜드1,사용성7,색상2,서비스3,양1,제품일반18,품질4",14,19,2


In [ ]:
# #트레인 셋에 없는 product number 제거
# test.reset_index(drop = True, inplace = True)
# for j in range(len(test)):
#   count = 0
#   for i in range(len(train)):
#     if train['product number'][i] == test['product number'][j]:
#       count += 1
#   if count == 0:
#     print(test['product number'][j])    
#     test = test.drop(j, axis = 0)

# test.reset_index(drop = True, inplace = True)

In [ ]:
test = test[test['sales_rank'] <= 50]
test

,index,product number,product name,date,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,remain_sales_time,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,five_star_per,four_star_per,three_star_per,two_star_per,one_star_per,functional_quality_name,functional_quality_per,functional_quality_opinion,review_content,remain_sales_limit,day,weekday
58264,697041,3.486486e+09,최종64만/한컴+마우스/HP 15s-fq2013TU 가성비 노트북/11세대 i5/2...,2021-05-19,디지털,노트북/PC/태블릿,노트북,1.0,234,0일 0시간,0,0.0,760000,5,HP총판이텍컴퓨터,1.0,고객서비스 우수,95.5,97.5,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,2
58265,697042,2.884596e+09,고부기 갤럭시 워치 액티브2 3D 포밍 풀커버 액정보호필름 2매,2021-05-19,디지털,노트북/PC/태블릿,액세서리,2.0,293,3일 01시간,1,107.0,8900,10,고부기몰,1.0,고객서비스 우수,88.4,100.0,4.0,4.4,59%,29%,7%,4%,1%,"성능,무게,배터리,배송속도","73%,66%,64%,49%","좋아요,가벼워요,보통이에요,빨라요","사용성8,제품일반23,크기1,품질3",3,19,2
58266,697043,2.817019e+09,[쿠폰15%]공식파트너 삼성 갤럭시탭S6 라이트 64GB 10.4 SM-P610 W...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,3.0,1019,3일 01시간,0,563.0,405900,10,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,4.6,72%,22%,4%,1%,1%,"성능,무게,배터리,배송속도","70%,84%,61%,54%","좋아요,보통이에요,오래가요,빨라요","가격57,구매12,디자인31,사용성107,상품평3,색상25,선호도1,성능34,양15...",3,19,2
58267,697044,3.411387e+09,[쿠폰16%]공식파트너 삼성 갤럭시탭S7 네이비 11.0 SM-T870 512G L...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,4.0,403,1일 01시간,0,292.0,999400,-1,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,5.0,95%,4%,1%,0%,0%,"성능,무게,배터리,배송속도","92%,99%,56%,72%","좋아요,보통이에요,오래가요,빨라요","가격20,구매9,디자인29,무게2,사용성81,색상30,성능49,양36,일반1,제품일...",1,19,2
58268,697045,2.540957e+09,[쿠폰15%할인]공식파트너 삼성 갤럭시탭A 8.0 2019 SM-T295 32GB/...,2021-05-19,디지털,노트북/PC/태블릿,태블릿,5.0,484,1일 01시간,0,206.0,168300,10,삼성공식파트너엠피나비,1.0,NaN,83.9,100.0,4.0,4.6,67%,21%,10%,1%,1%,"성능,무게,배터리,배송속도","83%,68%,60%,50%","좋아요,보통이에요,오래가요,빨라요","가격29,구매2,디자인4,무게2,사용성37,상품평1,성능18,양3,제품일반62,크기...",1,19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58899,697918,3.481584e+09,LG전자 27인치 모니터 27MN60T(68.58cm(27형)/IPS 패널/와이드1...,2021-05-19,디지털,모니터/프린터,모니터/주변기기,47.0,1,1일 02시간,0,0.0,199000,16,moduba,NaN,NaN,100.0,100.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,19,2
58900,697919,1.623036e+09,국산 모니터 받침대 5종 PC받침대,2021-05-19,디지털,모니터/프린터,모니터/주변기기,48.0,1,12일 02시간,0,119.0,4500,8,D_STORE,NaN,고객서비스 우수,95.4,100.0,3.0,4.4,57%,30%,8%,3%,2%,"화질,배송속도","76%,52%","볼만해요,빨라요","가격16,구매2,디자인4,사용성11,색상1,제품일반33,크기4,품질5",12,19,2
58901,697920,3.395649e+09,[어드밴스원] AF-270F200 보더리스 200Hz 프리싱크 게이밍 모니터/27,2021-05-19,디지털,모니터/프린터,모니터/주변기기,49.0,1,5일 02시간,0,0.0,185000,7,어드밴스원,1.0,고객서비스 우수,96.2,100.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,19,2
58902,697921,3.282551e+09,Fujitsu iX1600 북스캐너/양면스캔+우체국특송+정품,2021-05-19,디지털,모니터/프린터,프린터/복합기,50.0,22,3일 02시간,0,8.0,598000,3,씨게이트공식스토어,1.0,고객서비스 우수,97.7,100.0,4.0,5.0,100%,0%,0%,0%,0%,"성능,배송속도","88%,100%","좋아요,빨라요","가격2,구매1,디자인1,사용성4,색상1,양2,제품일반4,크기2,품질3",3,19,2


In [ ]:
'''
daily_info2 = daily_info.drop(columns = ['index', 'date', 'product number', 'product name', 'sales_rank',
                                        'remain_sales_time', 'supplier_name',
                                        'five_star_per', 'four_star_per', 
                                        'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 
                                        'functional_quality_per', 'functional_quality_opinion', 'review_content',
                                        'total_sales', 'product_large_type', 'review_num', 'official_store', 'customer_service', 'supplier_pos_review',
                                         'supplier_response_per', 'supplier_sales_performance', 'customer_total_satisfaction',
                                         'free_delivery', 'price', 'discount_per', 'remain_sales_limit', 'day', 'weekday'])
'''
'''
#baseline
daily_info2 = daily_info.drop(columns = ['index', 'date', 'product number', 'product name', 'sales_rank',
                                        'remain_sales_time', 'supplier_name',
                                        'five_star_per', 'four_star_per', 
                                        'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 
                                        'functional_quality_per', 'functional_quality_opinion', 'review_content',
                                        'total_sales', 'product_large_type', 'review_num', 'official_store', 'customer_service', 'supplier_pos_review',
                                         'supplier_response_per', 'supplier_sales_performance', 'customer_total_satisfaction'
                                         ])
'''
'''
#baseline + retailer info
daily_info2 = daily_info.drop(columns = ['index', 'date', 'product number', 'product name', 'sales_rank',
                                        'remain_sales_time', 'supplier_name',
                                        'five_star_per', 'four_star_per', 
                                        'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 
                                        'functional_quality_per', 'functional_quality_opinion', 'review_content',
                                        'total_sales', 'product_large_type', 'review_num',
                                        'customer_total_satisfaction'
                                         ])

'''
'''
#baseline + user generatged info
daily_info2 = daily_info.drop(columns = ['index', 'date', 'product name', 'sales_rank',
                                        'remain_sales_time', 'supplier_name',
                                        'five_star_per', 'four_star_per', 
                                        'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 
                                        'functional_quality_per', 'functional_quality_opinion', 'review_content',
                                         'product_large_type', 'official_store', 'customer_service', 'supplier_pos_review',
                                         'supplier_response_per', 'supplier_sales_performance'
                                         ])

'''

#baseline + retailer info + user generated info
daily_info2 = daily_info.drop(columns = ['index', 'date', 
                                        'remain_sales_time', 'product name',
                                        'five_star_per', 'four_star_per',
                                        'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 
                                        'functional_quality_per', 'functional_quality_opinion', 'review_content'
                                         ])

daily_info2

,product number,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
0,2.831097e+09,디지털,노트북/PC/태블릿,액세서리,1.0,1896,1,277.0,29700,15,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,4,28,6
1,2.247584e+09,디지털,노트북/PC/태블릿,액세서리,2.0,3624,0,896.0,165000,-1,Apple공인인증판매점,1.0,고객서비스 우수,89.6,100.0,5.0,4.8,6,28,6
2,3.012948e+09,디지털,노트북/PC/태블릿,액세서리,3.0,827,1,104.0,29700,12,비스비,1.0,고객서비스 우수,94.8,100.0,4.0,4.7,2,28,6
3,2.820897e+09,디지털,노트북/PC/태블릿,태블릿,4.0,1520,0,425.0,428450,5,삼성전자공식파트너,1.0,고객서비스 우수,90.2,100.0,4.0,4.8,10,28,6
4,2.961319e+09,디지털,노트북/PC/태블릿,태블릿,5.0,887,0,175.0,1174630,5,삼성공식파트너엠피나비,1.0,고객서비스 우수,80.1,100.0,4.0,4.8,3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58966,3.234159e+09,디지털,모니터/프린터,프린터/복합기,122.0,206,0,77.0,229000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.8,14,19,2
58967,2.904190e+09,디지털,모니터/프린터,프린터/복합기,123.0,47,0,52.0,169000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.5,14,19,2
58968,2.904112e+09,디지털,모니터/프린터,프린터/복합기,124.0,30,0,20.0,309000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.4,14,19,2
58969,2.904196e+09,디지털,모니터/프린터,프린터/복합기,125.0,71,0,49.0,219000,2,삼성전자공식몰,1.0,고객서비스 우수,90.2,100.0,3.0,4.6,14,19,2


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

c = daily_info2.columns[daily_info2.dtypes == object]

for i in c:
    daily_info2[i] = le.fit_transform(list(daily_info2[i]))

daily_info2

,product number,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
0,2.831097e+09,0,0,11,1.0,1896,1,277.0,29700,15,173,1.0,1,94.8,100.0,4.0,4.7,4,28,6
1,2.247584e+09,0,0,11,2.0,3624,0,896.0,165000,-1,4,1.0,1,89.6,100.0,5.0,4.8,6,28,6
2,3.012948e+09,0,0,11,3.0,827,1,104.0,29700,12,173,1.0,1,94.8,100.0,4.0,4.7,2,28,6
3,2.820897e+09,0,0,16,4.0,1520,0,425.0,428450,5,209,1.0,1,90.2,100.0,4.0,4.8,10,28,6
4,2.961319e+09,0,0,16,5.0,887,0,175.0,1174630,5,190,1.0,1,80.1,100.0,4.0,4.8,3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58966,3.234159e+09,0,1,17,122.0,206,0,77.0,229000,2,207,1.0,1,90.2,100.0,3.0,4.8,14,19,2
58967,2.904190e+09,0,1,17,123.0,47,0,52.0,169000,2,207,1.0,1,90.2,100.0,3.0,4.5,14,19,2
58968,2.904112e+09,0,1,17,124.0,30,0,20.0,309000,2,207,1.0,1,90.2,100.0,3.0,4.4,14,19,2
58969,2.904196e+09,0,1,17,125.0,71,0,49.0,219000,2,207,1.0,1,90.2,100.0,3.0,4.6,14,19,2


In [ ]:
train2 = daily_info2[:a[0]]
train2

,product number,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
0,2.831097e+09,0,0,11,1.0,1896,1,277.0,29700,15,173,1.0,1,94.8,100.0,4.0,4.7,4,28,6
1,2.247584e+09,0,0,11,2.0,3624,0,896.0,165000,-1,4,1.0,1,89.6,100.0,5.0,4.8,6,28,6
2,3.012948e+09,0,0,11,3.0,827,1,104.0,29700,12,173,1.0,1,94.8,100.0,4.0,4.7,2,28,6
3,2.820897e+09,0,0,16,4.0,1520,0,425.0,428450,5,209,1.0,1,90.2,100.0,4.0,4.8,10,28,6
4,2.961319e+09,0,0,16,5.0,887,0,175.0,1174630,5,190,1.0,1,80.1,100.0,4.0,4.8,3,28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58259,1.374893e+09,0,1,17,114.0,5,0,78.0,379000,2,186,1.0,1,96.0,100.0,4.0,4.4,1,18,1
58260,1.323999e+09,0,1,17,115.0,2,0,406.0,179000,2,187,1.0,1,95.2,100.0,4.0,4.7,6,18,1
58261,2.904099e+09,0,1,17,117.0,13,0,7.0,219000,2,207,1.0,1,90.2,100.0,3.0,4.6,1,18,1
58262,2.630368e+09,0,1,17,118.0,41,0,87.0,99000,2,186,1.0,1,96.0,100.0,4.0,4.9,1,18,1


In [ ]:
test2 = daily_info2[a[0]:]
test2

,product number,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
58264,3.486486e+09,0,0,5,1.0,234,0,0.0,760000,5,16,1.0,1,95.5,97.5,5.0,5.0,0,19,2
58265,2.884596e+09,0,0,11,2.0,293,1,107.0,8900,10,85,1.0,1,88.4,100.0,4.0,4.4,3,19,2
58266,2.817019e+09,0,0,16,3.0,1019,0,563.0,405900,10,190,1.0,0,83.9,100.0,4.0,4.6,3,19,2
58267,3.411387e+09,0,0,16,4.0,403,0,292.0,999400,-1,190,1.0,0,83.9,100.0,4.0,5.0,1,19,2
58268,2.540957e+09,0,0,16,5.0,484,0,206.0,168300,10,190,1.0,0,83.9,100.0,4.0,4.6,1,19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58966,3.234159e+09,0,1,17,122.0,206,0,77.0,229000,2,207,1.0,1,90.2,100.0,3.0,4.8,14,19,2
58967,2.904190e+09,0,1,17,123.0,47,0,52.0,169000,2,207,1.0,1,90.2,100.0,3.0,4.5,14,19,2
58968,2.904112e+09,0,1,17,124.0,30,0,20.0,309000,2,207,1.0,1,90.2,100.0,3.0,4.4,14,19,2
58969,2.904196e+09,0,1,17,125.0,71,0,49.0,219000,2,207,1.0,1,90.2,100.0,3.0,4.6,14,19,2


In [ ]:
test2 = test2[test2['sales_rank'] <= 50]
test2

,product number,product_large_type,product_medium_type,product_small_type,sales_rank,total_sales,free_delivery,review_num,price,discount_per,supplier_name,official_store,customer_service,supplier_pos_review,supplier_response_per,supplier_sales_performance,customer_total_satisfaction,remain_sales_limit,day,weekday
58264,3.486486e+09,0,0,5,1.0,234,0,0.0,760000,5,16,1.0,1,95.5,97.5,5.0,5.0,0,19,2
58265,2.884596e+09,0,0,11,2.0,293,1,107.0,8900,10,85,1.0,1,88.4,100.0,4.0,4.4,3,19,2
58266,2.817019e+09,0,0,16,3.0,1019,0,563.0,405900,10,190,1.0,0,83.9,100.0,4.0,4.6,3,19,2
58267,3.411387e+09,0,0,16,4.0,403,0,292.0,999400,-1,190,1.0,0,83.9,100.0,4.0,5.0,1,19,2
58268,2.540957e+09,0,0,16,5.0,484,0,206.0,168300,10,190,1.0,0,83.9,100.0,4.0,4.6,1,19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58899,3.481584e+09,0,1,7,47.0,1,0,0.0,199000,16,71,NaN,0,100.0,100.0,2.0,5.0,1,19,2
58900,1.623036e+09,0,1,7,48.0,1,0,119.0,4500,8,8,NaN,1,95.4,100.0,3.0,4.4,12,19,2
58901,3.395649e+09,0,1,7,49.0,1,0,0.0,185000,7,252,1.0,1,96.2,100.0,3.0,5.0,5,19,2
58902,3.282551e+09,0,1,17,50.0,22,0,8.0,598000,3,235,1.0,1,97.7,100.0,4.0,5.0,3,19,2


In [ ]:
train2 = train2.drop(columns = ['sales_rank'])
test2 = test2.drop(columns = ['sales_rank'])

In [ ]:
'''
lgbm = LGBMRegressor(n_estimators = 10000, objective = 'MAPE')
kf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
result = 0
best_score = 0
#n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정

for train_index, valid_index in kf.split(train2):
    x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
    lgbm.fit(x_train, np.log(y_train), eval_set = (x_valid, np.log(y_valid)), verbose = 1000, early_stopping_rounds = 30, eval_metric = 'rmse')    
    best_score += np.min(lgbm.evals_result_['valid_0']['mape'])/5
    # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
    result += np.exp(lgbm.predict(test2))/5
'''

"\nlgbm = LGBMRegressor(n_estimators = 10000, objective = 'MAPE')\nkf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다\nresult = 0\nbest_score = 0\n#n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정\n\nfor train_index, valid_index in kf.split(train2):\n    x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]\n    y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]\n    lgbm.fit(x_train, np.log(y_train), eval_set = (x_valid, np.log(y_valid)), verbose = 1000, early_stopping_rounds = 30, eval_metric = 'rmse')    \n    best_score += np.min(lgbm.evals_result_['valid_0']['mape'])/5\n    # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여\n    result += np.exp(lgbm.predict(test2))/5\n"

In [ ]:
# #test셋 구성
# #stratify()
# #machine learning x -> deep learning or machine learning
# #lag feature
# kf = KFold(n_splits = 2, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
# cat = CatBoostRegressor(learning_rate = 0.1, iterations = 10000, eval_metric = 'MAPE', task_type = 'GPU')
# result = 0
# best_score = 0

# for train_index, valid_index in kf.split(train2):
#     x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
#     y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
#     #cat.fit(x_train, y_train, eval_set = (x_valid, y_valid), verbose = 1000, early_stopping_rounds = 100)
#     cat.fit(x_train, np.log(y_train), eval_set = (x_valid, np.log(y_valid)), verbose = 1000, early_stopping_rounds = 100)    
#     best_score += cat.best_score_['validation']['MAPE']/5
#     # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
#     #result += cat.predict(test2)/5
#     result += np.exp(cat.predict(test2))/5


In [ ]:
# #분류문제일 경우 stratify KFold (분류 비율에 맞춰서)
# from sklearn.model_selection import KFold
# from lightgbm import LGBMRegressor

# kf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
# lgbm = LGBMRegressor(n_estimators = 10000, num_leaves = 100, learning_rate = 0.15)
# result = 0
# best_score = 0
# #n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정

# for train_index, valid_index in kf.split(train2):
#     x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
#     y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
#     lgbm.fit(x_train, np.log(y_train), eval_set = (x_valid, np.log(y_valid)), verbose = 25, early_stopping_rounds = 30, eval_metric = 'mape')    
#     best_score += np.min(lgbm.evals_result_['valid_0']['mape'])/5
#     # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
#     result += np.exp(lgbm.predict(test2))/5

Training until validation scores don't improve for 30 rounds.
[25]	valid_0's l2: 0.221746	valid_0's mape: 0.103569
[50]	valid_0's l2: 0.167475	valid_0's mape: 0.0873091
[75]	valid_0's l2: 0.14684	valid_0's mape: 0.0809278
[100]	valid_0's l2: 0.134435	valid_0's mape: 0.0765574
[125]	valid_0's l2: 0.124553	valid_0's mape: 0.0732741
[150]	valid_0's l2: 0.117495	valid_0's mape: 0.0705891
[175]	valid_0's l2: 0.112078	valid_0's mape: 0.0684383
[200]	valid_0's l2: 0.107749	valid_0's mape: 0.0667585
[225]	valid_0's l2: 0.103247	valid_0's mape: 0.0648977
[250]	valid_0's l2: 0.100203	valid_0's mape: 0.0635589
[275]	valid_0's l2: 0.0972499	valid_0's mape: 0.0622736
[300]	valid_0's l2: 0.0945507	valid_0's mape: 0.0610668
[325]	valid_0's l2: 0.0924905	valid_0's mape: 0.060118
[350]	valid_0's l2: 0.0907907	valid_0's mape: 0.0592946
[375]	valid_0's l2: 0.0896416	valid_0's mape: 0.0586422
[400]	valid_0's l2: 0.088134	valid_0's mape: 0.0578935
[425]	valid_0's l2: 0.0867549	valid_0's mape: 0.0572115
[45

In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor

kf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
lgbm = LGBMRegressor(n_estimators = 10000, num_leaves = 70, learning_rate = 0.1)
result = 0
best_score = 0
#n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정

for train_index, valid_index in kf.split(train2):
    x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
    lgbm.fit(x_train, np.log(y_train), eval_set = (x_valid, np.log(y_valid)), verbose = 25, early_stopping_rounds = 30, eval_metric = 'mape')    
    best_score += np.min(lgbm.evals_result_['valid_0']['mape'])/5
    # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
    result += np.exp(lgbm.predict(test2))/5

Training until validation scores don't improve for 30 rounds.
[25]	valid_0's l2: 0.2819	valid_0's mape: 0.120908
[50]	valid_0's l2: 0.215201	valid_0's mape: 0.101669
[75]	valid_0's l2: 0.186864	valid_0's mape: 0.0934853
[100]	valid_0's l2: 0.169836	valid_0's mape: 0.0883834
[125]	valid_0's l2: 0.156603	valid_0's mape: 0.0842164
[150]	valid_0's l2: 0.147862	valid_0's mape: 0.0814038
[175]	valid_0's l2: 0.140567	valid_0's mape: 0.0790341
[200]	valid_0's l2: 0.13406	valid_0's mape: 0.0768019
[225]	valid_0's l2: 0.129257	valid_0's mape: 0.0750858
[250]	valid_0's l2: 0.124857	valid_0's mape: 0.0733272
[275]	valid_0's l2: 0.120886	valid_0's mape: 0.0718217
[300]	valid_0's l2: 0.116939	valid_0's mape: 0.0703965
[325]	valid_0's l2: 0.113738	valid_0's mape: 0.0690981
[350]	valid_0's l2: 0.110909	valid_0's mape: 0.0680102
[375]	valid_0's l2: 0.108503	valid_0's mape: 0.0670605
[400]	valid_0's l2: 0.106033	valid_0's mape: 0.06604
[425]	valid_0's l2: 0.103995	valid_0's mape: 0.0652215
[450]	valid_0

In [ ]:
import xgboost
kf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08)
result = 0
best_score = 0
#n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정

for train_index, valid_index in kf.split(train2):
    x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
    y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
    xgb.fit(x_train, np.log(y_train), eval_set = [(x_valid, np.log(y_valid)), verbose = 25, eval_metric = 'mape')    
    best_score += np.min(lgbm.evals_result_['valid_0']['mape'])/5
    # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
    result += np.exp(xgb.predict(test2))/5

XGBoostError: ignored

In [ ]:
result.shape

In [ ]:
#lgbm = LGBMRegressor(n_estimators = 1000, n_jobs = -1, objective = 'mape')
#lgbm.fit(train2, train['sales_rank'], verbose = 1000)

In [ ]:
#cat = CatBoostRegressor(learning_rate = 0.1, iterations = 150000, loss_function = 'MAPE', task_type = 'GPU')
#cat.fit(train2, train['total_sales'], verbose = 1000, early_stopping_rounds = 30)

In [ ]:

# from sklearn.metrics import mean_squared_error
# from sklearn.ensemble import RandomForestRegressor
# kf = KFold(n_splits = 5, shuffle = True, random_state = 42) # n_splits는 높을수록 좋다
# rf = RandomForestRegressor(n_jobs = -1, random_state = 42) #n_estimator
# result = 0
# best_score = 0
# #n_estimators 기본값 100, early_stopping_rounds로 두면 어차피 멈추니깐 n_estimator 높게 설정
# train2 = train2.fillna(-1)

# for train_index, valid_index in kf.split(train2):
#     x_train, x_valid = train2.iloc[train_index], train2.iloc[valid_index]
#     y_train, y_valid = train['sales_rank'].iloc[train_index], train['sales_rank'].iloc[valid_index]
#     rf.fit(x_train, y_train)    
#     # 5번 교차검증할건데 각각의 시행에서 0.2의 가중치 부여
#     result_valid = rf.predict(x_valid)
#     best_score += np.sqrt(mean_squared_error(y_valid, result_valid))/5
#     print('split = ', best_score)


In [ ]:
#cat.best_score_

In [ ]:
#l2는 mean squared error
#np.min(cat.evals_result_['valid_0']['rmse'])
cat.best_score_['validation']['MAPE']

In [ ]:
best_score

In [ ]:
a = result - np.array(test['sales_rank'])
a

In [ ]:
result.shape

In [ ]:
result

In [ ]:
np.array(test['sales_rank']).shape

In [ ]:
result.shape

In [ ]:
r2_score(np.array(test['sales_rank']), result)
#R2 경향성 (0-1), 0이면 예측력이 없다, 1이면 예측력이 없다. (0.3이상이어야지 예측력이 있다)
#feature importance, permutation importance(어떤 칼럼이 중요한지, +,-)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

np.sqrt(mean_squared_error(np.array(test['sales_rank']), result))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

np.sqrt(mean_squared_error(np.array(test['total_sales']), result))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

importances = pd.Series(cat.feature_importances_, index = daily_info2.columns).sort_values(ascending = False)
plt.figure(figsize = (16, 8))
sns.barplot(importances.values, importances.index)

In [ ]:
#pi = PermutationImportance(lgbm), random_state = 42).fit(daily_info2, daily_info['total_sales'])
pi = PermutationImportance(cat, random_state = 42).fit(daily_info2, daily_info['sales_rank'])
pi